# Repeated Property IDs

## 1. Understanding the Data Structure

Before analyzing the "duplicates", we must understand what each column represents:

| Column | Meaning | Type |
|--------|---------|------|
| `id` | **Property identifier** — identifies the physical building/land | Identifier (not a feature) |
| `date` | **Sale date** — when the transaction occurred | Temporal |
| `price` | **Sale price** — the transaction amount | Target variable |
| Other columns | Property characteristics | Features |

**Critical distinction**: The `id` identifies **properties**, not **sales**. A property can be sold multiple times, generating multiple records with the same `id` but different `date` and `price` values.

## 2. Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import kagglehub
from pathlib import Path

path = kagglehub.dataset_download("harlfoxem/housesalesprediction")
csv_path = Path(path) / "kc_house_data.csv"
df = pd.read_csv(csv_path)
print(f"Dataset shape: {df.shape}")

Dataset shape: (21613, 21)


## 3. Identifying Repeated IDs

In [ ]:
id_counts = df["id"].value_counts()
repeated_ids = id_counts[id_counts > 1]

print(f"Total records: {len(df):,}")
print(f"Unique property IDs: {df['id'].nunique():,}")
print(f"Properties with multiple sales: {len(repeated_ids):,}")
print(f"Records from repeated IDs: {df['id'].isin(repeated_ids.index).sum():,}")

Total records: 21,613
Unique property IDs: 21,436
Properties with multiple sales: 176
Records from repeated IDs: 353


In [ ]:
# As percentages
repeated_records = df[df["id"].isin(repeated_ids.index)]
pct_repeated_records = 100 * len(repeated_records) / len(df)
pct_repeated_ids = 100 * len(repeated_ids) / df["id"].nunique()

print(f"\nPercentage of records with repeated IDs: {pct_repeated_records:.2f}%")
print(f"Percentage of properties sold more than once: {pct_repeated_ids:.2f}%")


Percentage of records with repeated IDs: 1.63%
Percentage of properties sold more than once: 0.82%


In [ ]:
# How many times are IDs repeated?
id_counts.value_counts().sort_index()

count
1    21260
2      175
3        1
Name: count, dtype: int64

## 4. What Do These Repeated Records Represent?

Let's examine specific examples to understand what differs between records with the same ID.

In [ ]:
example_ids = repeated_ids.head(3).index.tolist()

for prop_id in example_ids:
    print(f"\n{'='*70}")
    print(f"Property ID: {prop_id}")
    print("="*70)
    records = df[df["id"] == prop_id].sort_values("date")
    display(records[["id", "date", "price", "sqft_living", "bedrooms", "bathrooms", "grade"]])


Property ID: 795000620


,id,date,price,sqft_living,bedrooms,bathrooms,grade
17602,795000620,20140924T000000,115000.0,1080,3,1.0,5
17603,795000620,20141215T000000,124000.0,1080,3,1.0,5
17604,795000620,20150311T000000,157000.0,1080,3,1.0,5



Property ID: 3323059027


,id,date,price,sqft_living,bedrooms,bathrooms,grade
6630,3323059027,20140528T000000,326000.0,1720,3,2.75,7
6631,3323059027,20150225T000000,340000.0,1720,3,2.75,7



Property ID: 1450100390


,id,date,price,sqft_living,bedrooms,bathrooms,grade
10272,1450100390,20140905T000000,125000.0,920,3,1.0,6
10273,1450100390,20150316T000000,208000.0,920,3,1.0,6


### 4.1 Analysis: What Changes Between Records?

For properties sold multiple times, we expect:
- **`date`**: Always different (different transaction dates)
- **`price`**: Usually different (market conditions, negotiation)
- **Physical features**: Mostly unchanged (same building)

Let's verify this systematically:

In [ ]:
# Analyze what columns change between sales of the same property
feature_cols = [col for col in df.columns if col not in ["id", "date", "price"]]

properties_with_changes = 0
changed_features_count = {col: 0 for col in feature_cols}

for prop_id in repeated_ids.index:
    records = df[df["id"] == prop_id]
    
    has_any_change = False
    for col in feature_cols:
        if records[col].nunique() > 1:
            changed_features_count[col] += 1
            has_any_change = True
    
    if has_any_change:
        properties_with_changes += 1

print(f"Properties where features changed between sales: {properties_with_changes} / {len(repeated_ids)}")
print(f"\nFeatures that changed (and how often):")
for col, count in sorted(changed_features_count.items(), key=lambda x: -x[1]):
    if count > 0:
        print(f"  {col}: {count} properties")

Properties where features changed between sales: 0 / 176

Features that changed (and how often):


### 4.2 Conclusion: These Are Distinct Sale Transactions

The analysis confirms:

1. **Same property, different sales**: Each record represents a legitimate real estate transaction
2. Physical features remain mostly constant. Changes in features between sales represent renovations or updates, which are valid signal for the model.
3. **Price and date differ**: These are the transaction-specific values

**These are NOT duplicates in the data quality sense** — they are valid observations of different sale events.

## 5. The `id` Column: Why It Must Be Dropped

Regardless of repeated IDs, the `id` column **must be dropped** as a feature. Here's why:

### 5.1 Conceptual Reason: Identifiers Are Not Features

The `id` is a **property identifier** — an arbitrary label assigned to each building. It carries no intrinsic information about the property's value. Using it as a feature would be like using a person's social security number to predict their salary.

### 5.2 Technical Reason: Encoding Is Ineffective and prone to overfitting

With ~21,000 unique IDs:
- **One-hot encoding**: Would create ~21,000 binary features — computationally prohibitive and leads to extreme overfitting
- **Label encoding**: Would impose an artificial ordinal relationship (property 1000 is not "greater than" property 500)
- **Target encoding**: Would cause severe data leakage (encoding the ID with the target we're trying to predict)

**Advanced Alternatives (and why they fail here):**
In high-cardinality scenarios (e.g., Zip Codes), we often use advanced techniques. However, they require multiple examples per category to work:
- **Leave-One-Out (LOO) Encoding**: Calculates the mean target of *other* records with the same ID. Since most IDs appear only once, LOO would either return the global mean (no signal) or, for the few repeated IDs, perfectly leak the price of the *only* other sale, leading to massive overfitting.
- **Hash / CatBoost Encoding**: These also rely on recurrence. With `id` being almost unique per row, these techniques cannot extract a statistical pattern.

### 5.3 Practical Reason: The Model Would Memorize

If we included `id` as a feature, it creates a dangerous "shortcut" for the model:
- **Memorization vs. Learning**: The model can learn to identify specific houses (e.g., "if id == 123456...") instead of learning general rules (e.g., "if sqft > 2000...").
- **The "New ID" Problem**: While the model *might* still use other features, the ID rules become useless or harmful for any property not seen in the training set.
- **Result**: We get a model that performs great on the training data (by memorizing IDs) but fails on new data (where those IDs don't exist).

In [ ]:
# Demonstrate why ID can't be encoded
n_unique_ids = df["id"].nunique()
n_features = df.shape[1] - 2  # excluding id and price

print(f"Current number of features: {n_features}")
print(f"Number of unique IDs: {n_unique_ids:,}")
print(f"\nIf we one-hot encoded 'id':")
print(f"  New feature count: {n_features + n_unique_ids - 1:,}")
print(f"  Features would outnumber samples!")
print(f"\nThis is clearly not a viable approach.")

Current number of features: 19
Number of unique IDs: 21,436

If we one-hot encoded 'id':
  New feature count: 21,454
  Features would outnumber samples!

This is clearly not a viable approach.


## 6. Implications for Model Training

Now that we understand the data, let's consider the implications for model training.

### 6.1 The Non-Problem: Property ID "Leakage"

Some might worry: "If the same property appears in train and test sets, isn't that data leakage?"

**Answer: No, provided we respect time.**

If a property sold in 2014 (train) and again in 2015 (test), this is **not leakage**. In a real-world production scenario, when predicting the 2015 price, we *would* have access to the 2014 sale history. This mimics the actual information available at inference time.

**Note on Identification:**
While we drop the `id` column to prevent explicit memorization, the model might still "identify" properties via high-precision spatial coordinates (`lat`, `long`). This is called **spatial overfitting**, but it is distinct from the data leakage concern of using future information to predict the past.

In [ ]:
# Can properties be uniquely identified by their features alone?
feature_cols_for_id = ["sqft_living", "sqft_lot", "bedrooms", "bathrooms", 
                       "floors", "waterfront", "view", "condition", "grade",
                       "yr_built", "lat", "long"]

# Count unique feature combinations
unique_combinations = df.drop_duplicates(subset=feature_cols_for_id)

print(f"Total records: {len(df):,}")
print(f"Unique property IDs: {df['id'].nunique():,}")
print(f"Unique feature combinations: {len(unique_combinations):,}")
print(f"\nDifference: {df['id'].nunique() - len(unique_combinations):,} properties share features with others")

Total records: 21,613
Unique property IDs: 21,436
Unique feature combinations: 21,420

Difference: 16 properties share features with others


### 6.2 The Real Problem: Temporal Leakage

The actual concern with this dataset is **temporal leakage**, which affects ALL records, not just repeated IDs.

**The problem**: A random train/test split mixes sales from different time periods. The model can learn from May 2015 sales to predict January 2014 prices — using "future" information to predict the "past".

**The solution**: Use a **temporal split** — train on older sales, test on newer sales.

## 7. Should We Keep All Records?

Given that repeated IDs represent valid, distinct sales, should we keep all records?

### Arguments FOR Keeping All Records

1. **Each is a valid observation**: Different sales at different times with different prices
2. **Richer training data**: More examples for the model to learn from
3. **Captures market dynamics**: Price changes for the same property reflect market trends

### Arguments FOR Deduplication (and why they are mostly invalid here)

*Why might someone suggest keeping only the most recent record?*

1.  **Feature Collision (Same X, Different Y)**:
    *   *The Argument*: If physical features are static, the model sees the same house with two different prices.
    *   *The Flaw*: This is only true if we ignore **Time**. Since `date` is a crucial feature, the input $X$ (House + Date) is actually unique. The price difference is explained by the time difference.

2.  **Primacy of Recency**:
    *   *The Argument*: The most recent sale is the "true" current value.
    *   *The Flaw*: This applies to *appraisal* (what is it worth *now*?), but not necessarily to *machine learning training*. Old data helps the model learn how prices evolve over time. Unless the market has fundamentally broken/changed structure (e.g., pre- vs. post-housing crash), historical data is valuable signal, not noise.

### Recommendation

**Keep all records.**

For this specific dataset, deduplication is **suboptimal** and not a sound strategy.

1.  **The "Noise" is Signal**: The fact that the *same* house sold for different prices at different times forces the model to use the `date` feature to explain the difference. This helps the model learn market trends (inflation/deflation).
2.  **Short Time Horizon**: This dataset spans only 1 year. The "older" sale is not obsolete history; it's a recent data point that helps establish the price baseline.
3.  **Realistic Production Scenario**: If a property was sold in the training period AND the test period, this is realistic — we would actually know about the earlier sale when predicting the later one.

In [ ]:
# With temporal split: how many properties appear in both train and test?
df["date_parsed"] = pd.to_datetime(df["date"].str[:8], format="%Y%m%d")
df_sorted = df.sort_values("date_parsed")

split_idx = int(len(df_sorted) * 0.8)
train_df = df_sorted.iloc[:split_idx]
test_df = df_sorted.iloc[split_idx:]

train_ids = set(train_df["id"])
test_ids = set(test_df["id"])
shared_ids = train_ids.intersection(test_ids)

print(f"Temporal split (80/20):")
print(f"  Train: {len(train_df):,} records, {len(train_ids):,} unique properties")
print(f"  Test: {len(test_df):,} records, {len(test_ids):,} unique properties")
print(f"\nProperties appearing in BOTH sets: {len(shared_ids)}")
print(f"\nThis is REALISTIC: we know about past sales when predicting future ones.")

Temporal split (80/20):
  Train: 17,290 records, 17,203 unique properties
  Test: 4,323 records, 4,323 unique properties

Properties appearing in BOTH sets: 90

This is REALISTIC: we know about past sales when predicting future ones.


## 8. Summary and Recommendations

### What We Found

| Observation | Explanation |
|-------------|-------------|
| ~0.85% of records have repeated IDs | Same property sold multiple times |
| Physical features don't change | Expected — same building |
| Price and date differ | Different transactions |

### Recommended Actions

| Action | Rationale |
|--------|----------|
| **Drop `id` column** | It's an identifier, not a feature; cannot be meaningfully encoded |
| **Keep all records** | Each is a valid sale transaction |
| **Use temporal split** | Prevents temporal leakage (the real issue) |

### What NOT to Do

| Incorrect Approach | Why It's Wrong |
|-------------------|----------------|
| One-hot encode `id` | Creates ~21,000 features; leads to memorization |
| Remove "duplicate" records | Loses valid training data |
| Use `GroupShuffleSplit` by `id` | Doesn't address temporal leakage; adds complexity |
| Use random train/test split | Causes temporal leakage |

In [ ]:
print("""
FINAL PREPROCESSING GUIDANCE
============================

1. DROP the 'id' column:
   df = df.drop(columns=['id'])

2. KEEP all records (no deduplication needed)

3. Use TEMPORAL train/test split:
   df_sorted = df.sort_values('date')
   split_idx = int(len(df_sorted) * 0.8)
   train = df_sorted.iloc[:split_idx]
   test = df_sorted.iloc[split_idx:]

See notebook p3-03-temporal_leakage.ipynb for detailed explanation.
""")


FINAL PREPROCESSING GUIDANCE

1. DROP the 'id' column:
   df = df.drop(columns=['id'])

2. KEEP all records (no deduplication needed)

3. Use TEMPORAL train/test split:
   df_sorted = df.sort_values('date')
   split_idx = int(len(df_sorted) * 0.8)
   train = df_sorted.iloc[:split_idx]
   test = df_sorted.iloc[split_idx:]

See notebook p3-03-temporal_leakage.ipynb for detailed explanation.



## 9. Advanced Nuance: ID as a Key for Feature Engineering

A critical distinction must be made regarding the `id` column. While we established that `id` should not be used as a raw feature (categorical variable) to prevent memorization, it plays a different, vital role in real-world production systems: **as a Relational Key.**

### The Intuition
In a production Automated Valuation Model (AVM), the history of a specific asset is often the strongest predictor of its current value. If a house sold for \$500,000 last year, that is a powerful "anchor" for today's price.

### The Implementation: Lag Features
In a scenario with deep historical data, we would not drop `id`. Instead, we would use it to create **Lag Features**.
1. Sort data by `id` and `date`.
2. Shift the `price` column to create a `previous_price` feature.
3. Calculate the delta (`price` - `previous_price`).
4. **Then** drop the `id` column and train on the *change* in price.

### The Challenge: Signal Sparsity vs. Missing Values
You might wonder: *"Can't modern models (like XGBoost or LightGBM) handle missing values automatically?"*

**Yes, they can.** However, the issue here is not just technical (NaNs causing errors) but statistical (Signal-to-Noise). Let's quantify how much "signal" actually exists in this dataset.

In [ ]:
# Let's attempt to engineer the "Previous Price" feature 
# to see if it provides enough signal for the model.

# 1. Sort by ID and Date to ensure correct order
df_lag = df.sort_values(by=['id', 'date']).copy()

# 2. Create the lag feature: What was the price of this specific ID in the previous row?
df_lag['previous_price'] = df_lag.groupby('id')['price'].shift(1)

# 3. Calculate metrics
total_rows = len(df_lag)
rows_with_history = df_lag['previous_price'].notna().sum()
missing_history = df_lag['previous_price'].isna().sum()

print(f"Attempting to use ID for Lag Features:")
print(f"---------------------------------------")
print(f"Total records:               {total_rows:,}")
print(f"Records with history (Signal): {rows_with_history:,}  ({(rows_with_history/total_rows)*100:.2f}%)")
print(f"Records with NaN (Noise):    {missing_history:,}  ({(missing_history/total_rows)*100:.2f}%)")

Attempting to use ID for Lag Features:
---------------------------------------
Total records:               21,613
Records with history (Signal): 177  (0.82%)
Records with NaN (Noise):    21,436  (99.18%)


### Conclusion on Historical Features

As the calculation above shows, **99.18%** of our rows would have a `NaN` (missing value) for `previous_price`.

**The Verdict:**
1.  **Handling NaNs:** While modern models *can* handle NaNs (e.g., by sending them down a default branch in a tree), they still need a sufficient number of *non-NaN* examples to learn the underlying pattern.
2.  **Sparsity Risk:** With only ~176 examples of "how history predicts future price," the model is unlikely to learn a generalizable rule. It risks treating the feature as noise or overfitting to those specific 176 instances.
3.  **Production Reality:** In a real company database with 10 years of sales, this number might be 50% or 60%. In that case, **you would absolutely keep this feature.**

**Recommendation:** For this specific educational dataset, we drop `id` to focus on physical features. In a real-world project with denser history, using `id` to build lag features is a standard and powerful technique.